In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from birdCLEFloaddata import load_audiofile,load_metadata,get_melspectrogram
from birdCLEFFunctions import Dynamic_CNN, Dynamic_CNN2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import Resize
from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import Grayscale, ToPILImage
import torchvision
import soundfile as sf
import random
import math
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

plt.rcParams['figure.dpi'] = 600

In [5]:
path = r"D:\KU\Masters\AppML\APPML-BirdCLEF\data"
meta_data = load_metadata(path,trim=True)
meta_data.head(10)

,latitude,longitude,common_name,rating,filename
0,14.8294,-17.2303,Vitelline Masked-Weaver,4.0,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...
1,14.8294,-17.2303,Vitelline Masked-Weaver,5.0,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...
2,14.8294,-17.2303,Vitelline Masked-Weaver,3.5,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...
3,2.2662,31.5595,Vitelline Masked-Weaver,4.0,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...
4,2.2662,31.5595,Vitelline Masked-Weaver,3.0,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...
5,15.8874,-16.4867,Vitelline Masked-Weaver,4.0,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...
6,14.0606,-16.7575,Vitelline Masked-Weaver,3.5,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...
7,14.9101,-17.0724,Vitelline Masked-Weaver,3.5,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...
8,-2.8145,37.4113,Vitelline Masked-Weaver,3.5,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...
9,13.3198,-13.3906,Vitelline Masked-Weaver,3.0,D:\KU\Masters\AppML\APPML-BirdCLEF\data/train_...


### Get total time, average rating, and average time for each bird

In [9]:
total_time = {}
avg_rating = {}
avg_time = {}
i = 1
for bird in meta_data['common_name'].unique():
    print(i/len(meta_data['common_name'].unique()))
    audio_length = 0
    number_files = len(meta_data[meta_data['common_name'] == bird])
    avg_rating[bird] = meta_data[meta_data['common_name'] == bird]['rating'].sum()/number_files
    for file in  meta_data[meta_data['common_name'] == bird]['filename']:
        audio, sr = load_audiofile(file)
        audio_length += audio.shape[0]/sr
    total_time[bird] = audio_length
    avg_time[bird] = audio_length/number_files
    i += 1

Vitelline Masked-Weaver
Rock Martin
African Blue Flycatcher
Golden-backed Weaver
Chinspot Batis
Northern Gray-headed Sparrow
Hunter's Cisticola
Yellow-fronted Canary
White-throated Bee-eater
Scarlet-chested Sunbird
Red-chested Cuckoo
Rufous Chatterer
Brown-chested Alethe
Chestnut Sparrow
Helmeted Guineafowl
Blue-cheeked Bee-eater
Brubru
Common House-Martin
Red-fronted Barbet
African Gray Flycatcher
White-chinned Prinia
Black-throated Barbet
Quailfinch
Yellow-spotted Barbet
Hadada Ibis
Slender-billed Greenbul
Tawny-flanked Prinia
Hamerkop
Willow Warbler
Spur-winged Lapwing
Pin-tailed Whydah
Spectacled Weaver
Wood Sandpiper
Gray-backed Fiscal
Eurasian Hoopoe
Black-throated Apalis
Mouse-colored Penduline-Tit
Cabanis's Greenbul
Eastern Mountain Greenbul
Speckled Mousebird
Common Bulbul
Black Crake
Fan-tailed Raven
Little Weaver
Yellow-billed Duck
Green Woodhoopoe
Black-and-white-casqued Hornbill
Fork-tailed Drongo
Mariqua Sunbird
Black-backed Puffback
Yellow-billed Stork
Ring-necked Dove
D

In [12]:
#np.save('plot_data/total_time', total_time) 
#np.save('plot_data/avg_rating', avg_rating) 
#np.save('plot_data/avg_time', avg_time) 

### Get accuracy for each bird

In [23]:
val_probabilities = np.load('plot_data/val_probabilities.npy')
val_true_labels = np.load('plot_data/val_true_labels.npy')
val_predictions = np.argmax(val_probabilities, axis=1)

In [29]:
bird_accuracy = {}
for i, bird in enumerate(meta_data['common_name'].unique()):
    mask = val_true_labels == i+1
    bird_accuracy[bird] = np.sum(val_predictions[mask]==i+1)/np.sum(mask)


In [30]:
#np.save('plot_data/bird_accuracy', bird_accuracy) 